In [1]:
# Import scientific computing libraries.
import numpy as np
import pandas as pd
from pathlib import Path
import os
import re
from collections import Counter

In [2]:
# Load in directory containing all data files.
data_dir = Path('../SI_Student_Surveys_Full_Data')
csv_files = list(data_dir.glob('*/*.csv'))

In [3]:
# Initiate a list to hold all data files.
all_data = []
# Open each file and append to the list.
for file_path in sorted(csv_files):
    df = pd.read_csv(file_path, encoding='latin-1', header=1)
    df['source_file'] = file_path.stem
    all_data.append(df)


In [4]:
# Staple together every dataframe, such that observations from each student align neatly.
master_df = pd.concat(all_data, ignore_index=True)
print(f"Master dataframe shape: {master_df.shape}")
print(master_df.columns.tolist())

Master dataframe shape: (34515, 110)
['Start Date', 'End Date', 'Response Type', 'IP Address', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date', 'Response ID', 'External Data Reference', 'Location Latitude', 'Location Longitude', 'Distribution Channel', 'User Language', 'Overall Helpfulness of SI - Q1. My online SI session is helpful.', 'SI Techniques for Promoting Learning - Q2.  The opportunity to ask questions during the SI session is beneficial.', 'SI Techniques for Promoting Learning - Q3. Working practice problems in SI helps me learn concepts.', 'SI Techniques for Promoting Learning - Q4. Discussion of course concepts during the session increases my understanding of the concepts.', 'SI Leader Ability & Helpfulness - Q5. My SI Leader helps me better understand information from the lecture.', 'SI Leader Ability & Helpfulness - Q6. My SI Leader is willing to help students who are having difficulty with course content.', 'SI Leader Understanding of Subject Matter - Q

In [5]:
# Define columns to drop that do not contribute any significant information.
cols_to_drop = [
    'Leader Name',
    'Leader Username',
    'Course Code',
    'Drill ID',
    'Response Type',
    'IP Address',
    'Progress',
    'Finished',
    'External Data Reference',
    'Distribution Channel',
    'User Language',
    'Location Longitude',
    'Location Latitude',
    'course number',
    'Recipient First Name',
    'Recipient Last Name',
    'EXSC 3353 SI Feedback Survey Spring 2019'
]

master_df_clean = master_df.drop(columns=cols_to_drop, errors='ignore')

In [6]:
# Extract metadata columns.
metadata_cols = [
    'Start Date', 'End Date', 'Duration (in seconds)', 
    'Recorded Date', 'Response ID', 'source_file'
]

In [7]:
# Columns for questions Q1 through Q11 or 12 tend to be columns that have ratings.
rating_cols = [col for col in master_df_clean.columns 
               if col not in metadata_cols  # Exclude metadata
               and '?' not in col  # Exclude open-ended questions
               and any(keyword in col for keyword in [
                   'Helpfulness', 'Techniques for Promoting Learning', 
                   'Leader Ability', 'Leader Preparation', 'Leader Understanding',
                   'Subject Matter', 'Collaborative Activities', 
                   'Engagement in', 'Mastery of Subject', 'Drill Leader',
                   'Drill Techniques', 'Drop-in/Chat'
               ])]

# Text columns are consistently Q12, Q13, Q14, or Q15. 
text_cols = [col for col in master_df_clean.columns 
             if '?' in col or col.startswith(('Q12', 'Q13', 'Q14', 'Q15'))]

# Remove any text columns that got caught in ratings
rating_cols = [col for col in rating_cols if col not in text_cols]

print(f"Metadata columns: {metadata_cols}")
print(f"Rating columns: {rating_cols}")
print(f"Text columns: {text_cols}")

Metadata columns: ['Start Date', 'End Date', 'Duration (in seconds)', 'Recorded Date', 'Response ID', 'source_file']
Rating columns: ['Overall Helpfulness of SI - Q1. My online SI session is helpful.', 'SI Techniques for Promoting Learning - Q2.  The opportunity to ask questions during the SI session is beneficial.', 'SI Techniques for Promoting Learning - Q3. Working practice problems in SI helps me learn concepts.', 'SI Techniques for Promoting Learning - Q4. Discussion of course concepts during the session increases my understanding of the concepts.', 'SI Leader Ability & Helpfulness - Q5. My SI Leader helps me better understand information from the lecture.', 'SI Leader Ability & Helpfulness - Q6. My SI Leader is willing to help students who are having difficulty with course content.', 'SI Leader Understanding of Subject Matter - Q7. My SI Leader demonstrates competence in his/her subject.', 'SI Leader Preparation & Planning - Q8. My SI Leader comes to my SI session knowing what wa

In [8]:
# Ensure that every column has been categorized as rating or text.
all_categorized = set(metadata_cols + rating_cols + text_cols)
all_columns = set(master_df_clean.columns)
missing_cols = all_columns - all_categorized

print(f"Total columns: {len(master_df_clean.columns)}")
print(f"Categorized: {len(all_categorized)}")
print(f"Missing columns: {len(missing_cols)}")
if missing_cols:
    print("Uncategorized columns:")
    for col in missing_cols:
        print(f"  - {col}")

Total columns: 93
Categorized: 93
Missing columns: 0


In [9]:
# Create an empty list to group the themes of different questions together in logical formats.
# Some questions have slightly different wording, but are otherwise asking the same things.
question_consolidation = {}


# Group 1: Overall Helpfulness of Sessions
for col in rating_cols:
    if any(x in col for x in ['My online SI session is helpful', 'My SI session is helpful', 
                               'My SI sessions have been very helpful', 'My drill session is helpful']):
        question_consolidation[col] = 'Overall_Session_Helpfulness'

# Group 2: Blackboard/Course Folder
for col in rating_cols:
    if 'Blackboard' in col and 'folder' in col.lower() or 'Blackboard course' in col:
        question_consolidation[col] = 'Blackboard_Resources'

# Group 3: Weekly Emails
for col in rating_cols:
    if 'weekly emails' in col.lower():
        question_consolidation[col] = 'Weekly_Emails'

# Group 4: Drop-in/Chat Tutoring
for col in rating_cols:
    if 'Drop-in' in col or 'Chat Economics' in col:
        question_consolidation[col] = 'Dropin_Tutoring'

# Group 5: Opportunity to Ask Questions
for col in rating_cols:
    if 'opportunity to ask questions' in col.lower():
        question_consolidation[col] = 'Opportunity_Ask_Questions'

# Group 6: Working Practice Problems
for col in rating_cols:
    if 'practice problems' in col.lower() or 'Working practice' in col:
        question_consolidation[col] = 'Practice_Problems'

# Group 7: Discussion/Understanding Concepts
for col in rating_cols:
    if any(x in col for x in ['Discussion of course concepts', 'Discussing course concepts',
                               'explained differently', 'increases my understanding']):
        question_consolidation[col] = 'Concept_Discussion_Understanding'

# Group 8: Leader Helps Understanding
for col in rating_cols:
    if 'helps me better understand information from the lecture' in col.lower():
        question_consolidation[col] = 'Leader_Helps_Understanding'

# Group 9: Leader Willing to Help
for col in rating_cols:
    if 'willing to help students who are having difficulty' in col.lower():
        question_consolidation[col] = 'Leader_Willing_To_Help'

# Group 10: Leader Subject Competence
for col in rating_cols:
    if any(x in col for x in ['demonstrates competence', 'competent in his/her subject',
                               'Mastery of Subject Matter']):
        question_consolidation[col] = 'Leader_Subject_Competence'

# Group 11: Leader Knows Class Content
for col in rating_cols:
    if 'knowing what was covered in class' in col.lower():
        question_consolidation[col] = 'Leader_Knows_Class_Content'

# Group 12: Leader Has Plan
for col in rating_cols:
    if 'developed plan' in col.lower() or 'clear path' in col.lower():
        question_consolidation[col] = 'Leader_Has_Plan'

# Group 13: Collaborative Activities
for col in rating_cols:
    if 'incorporates collaborative activities' in col.lower():
        question_consolidation[col] = 'Collaborative_Activities'

# Group 14: Online Engagement
for col in rating_cols:
    if 'engaging content' in col.lower():
        question_consolidation[col] = 'Online_Engaging_Content'

# Group 15: Online Participation
for col in rating_cols:
    if 'actively participate in my online sessions' in col.lower():
        question_consolidation[col] = 'Online_Active_Participation'

# Group T1: How collaboration contributed to understanding
for col in text_cols:
    if 'working collaboratively' in col.lower() and 'understanding of course concepts' in col.lower():
        question_consolidation[col] = 'Text_Collaboration_Understanding'

# Group T2: Suggestions for leader performance
for col in text_cols:
    if 'suggestions' in col.lower() and 'leader' in col.lower() and 'performance' in col.lower():
        question_consolidation[col] = 'Text_Leader_Performance_Suggestions'

# Group T3: Suggestions for SI Program overall
for col in text_cols:
    if 'suggestions' in col.lower() and ('SI Program overall' in col or 'MEEG Drill Program overall' in col):
        question_consolidation[col] = 'Text_Program_Overall_Suggestions'

# Group T4: Suggestions for online SI experience
for col in text_cols:
    if 'online' in col.lower() and 'experience' in col.lower() and 'suggestions' in col.lower():
        question_consolidation[col] = 'Text_Online_Experience_Suggestions'

# Group T5: Other suggestions, or does not fit into everything else.
for col in text_cols:
    if 'OTHER suggestions' in col:
        question_consolidation[col] = 'Text_Other_Suggestions'

# Check coverage.
mapped_count = len(question_consolidation)
total_questions = len(rating_cols) + len(text_cols)
print(f"Mapped: {mapped_count}/{total_questions} questions")
print(f"Unique questions: {len(set(question_consolidation.values()))}")

# Show unmapped columns.
all_cols = rating_cols + text_cols
unmapped = [col for col in all_cols if col not in question_consolidation]
if unmapped:
    print(f"Unmapped columns ({len(unmapped)}):")
    for col in unmapped:
        print(f"  - {col}")

# Show consolidation groups
consolidation_counts = Counter(question_consolidation.values())
print("\nConsolidation groups:")
for new_col, count in sorted(consolidation_counts.items()):
    print(f"  {new_col}: {count} variations")

Mapped: 87/87 questions
Unique questions: 20

Consolidation groups:
  Blackboard_Resources: 3 variations
  Collaborative_Activities: 6 variations
  Concept_Discussion_Understanding: 9 variations
  Dropin_Tutoring: 1 variations
  Leader_Has_Plan: 7 variations
  Leader_Helps_Understanding: 7 variations
  Leader_Knows_Class_Content: 5 variations
  Leader_Subject_Competence: 7 variations
  Leader_Willing_To_Help: 7 variations
  Online_Active_Participation: 1 variations
  Online_Engaging_Content: 1 variations
  Opportunity_Ask_Questions: 5 variations
  Overall_Session_Helpfulness: 4 variations
  Practice_Problems: 4 variations
  Text_Collaboration_Understanding: 5 variations
  Text_Leader_Performance_Suggestions: 7 variations
  Text_Online_Experience_Suggestions: 1 variations
  Text_Other_Suggestions: 1 variations
  Text_Program_Overall_Suggestions: 4 variations
  Weekly_Emails: 2 variations


In [10]:
# Show all variations for each consolidated question
from collections import defaultdict

# Group by consolidated name
consolidation_groups = defaultdict(list)
for original_col, consolidated_name in question_consolidation.items():
    consolidation_groups[consolidated_name].append(original_col)

# Display each group
for consolidated_name in sorted(consolidation_groups.keys()):
    print(f"{consolidated_name} ({len(consolidation_groups[consolidated_name])} kinds of questions.)")
    for i, original in enumerate(consolidation_groups[consolidated_name], 1):
        print(f"{i}. {original}")

Blackboard_Resources (3 kinds of questions.)
1. Overall Helpfulness of SI - Q2. Having the SI folder in my course Blackboard is helpful.
2. Overall Helpfulness of SI - Q2. Having a Blackboard course for my SI session is helpful.
3. Overall Helpfulness - Q2. Having the SI folder in my course Blackboard is helpful.
Collaborative_Activities (6 kinds of questions.)
1. SI Leader's Use of Collaborative Activities - Q10. My SI Leader incorporates collaborative activities into our sessions.
2. SI Leader's Use of Collaborative Activities - Q10. The SI Leader incorporates collaborative activities into our sessions.
3. SI Leader's Use of Collaborative Activities - Q11. My SI Leader incorporates collaborative activities into our sessions.
4. Drill Leader's Use of Collaborative Activities - Q9. My drill leader incorporates collaborative activities into our sessions.
5. SI Leader's Use of Collaborative Activities - Q12. The SI Leader incorporates collaborative activities into our sessions.
6. SI Lea

In [11]:
# Create better column names with R_ or T_ prefix
renamed_consolidation = {}

for original_col, consolidated_name in question_consolidation.items():
    # Add prefix based on type.
    if consolidated_name.startswith('Text_'):
        # Remove 'Text_' and add 'T_' prefix.
        new_name = 'T_' + consolidated_name.replace('Text_', '')
    else:
        # Add 'R_' prefix for ratings.
        new_name = 'R_' + consolidated_name
    
    renamed_consolidation[original_col] = new_name

# Create consolidated dataframe with better names.
consolidated_df = master_df_clean[metadata_cols].copy()

for new_col_name in sorted(set(renamed_consolidation.values())):
    # Get all original columns that map to this new name.
    original_cols = [col for col, name in renamed_consolidation.items() if name == new_col_name]
    
    # Combine them using coalesce.
    consolidated_df[new_col_name] = master_df_clean[original_cols].bfill(axis=1).iloc[:, 0]

In [12]:
# Create a metadata file that maps new column names to their original variations
column_metadata = defaultdict(list)
for original_col, new_name in renamed_consolidation.items():
    column_metadata[new_name].append(original_col)

# Save the metadata mapping.
import json
metadata_export = {
    col_name: {
        'type': 'Rating' if col_name.startswith('R_') else 'Text',
        'variations': variations
    }
    for col_name, variations in column_metadata.items()
}

with open('../column_metadata.json', 'w') as f:
    json.dump(metadata_export, f, indent=2)

print(f"Original shape: {master_df_clean.shape}")
print(f"Consolidated shape: {consolidated_df.shape}")
print(f"\nNew columns:")
print(f"  Metadata: {len(metadata_cols)}")
print(f"  Rating (R_): {sum(1 for c in consolidated_df.columns if c.startswith('R_'))}")
print(f"  Text (T_): {sum(1 for c in consolidated_df.columns if c.startswith('T_'))}")

print("\nColumn metadata saved to ../column_metadata.json")
print("\nSample columns:")
for col in sorted([c for c in consolidated_df.columns if c.startswith(('R_', 'T_'))])[:10]:
    print(f"  {col}")

Original shape: (34515, 93)
Consolidated shape: (34515, 26)

New columns:
  Metadata: 6
  Rating (R_): 15
  Text (T_): 5

Column metadata saved to ../column_metadata.json

Sample columns:
  R_Blackboard_Resources
  R_Collaborative_Activities
  R_Concept_Discussion_Understanding
  R_Dropin_Tutoring
  R_Leader_Has_Plan
  R_Leader_Helps_Understanding
  R_Leader_Knows_Class_Content
  R_Leader_Subject_Competence
  R_Leader_Willing_To_Help
  R_Online_Active_Participation


In [13]:
consolidated_df

,Start Date,End Date,Duration (in seconds),Recorded Date,Response ID,source_file,R_Blackboard_Resources,R_Collaborative_Activities,R_Concept_Discussion_Understanding,R_Dropin_Tutoring,...,R_Online_Engaging_Content,R_Opportunity_Ask_Questions,R_Overall_Session_Helpfulness,R_Practice_Problems,R_Weekly_Emails,T_Collaboration_Understanding,T_Leader_Performance_Suggestions,T_Online_Experience_Suggestions,T_Other_Suggestions,T_Program_Overall_Suggestions
0,11/6/2019 9:03,11/6/2019 9:03,29,11/6/2019 9:03,R_2QlC06WEVwEXtjQ,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,Strongly agree,Neither disagree nor agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN
1,11/6/2019 10:06,11/6/2019 10:08,88,11/6/2019 10:08,R_VPgtpoWlVBH0E81,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,Strongly agree,Strongly disagree,Moderately agree,Moderately agree,NaN,NaN,NaN,It is extremely difficult to ask beneficial qu...,NaN,NaN
2,11/6/2019 12:18,11/6/2019 12:18,35,11/6/2019 12:18,R_sHdb8bWznRjZ6yB,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Moderately agree,NaN,...,Strongly agree,Unable to judge,Moderately agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN
3,11/6/2019 16:51,11/6/2019 16:52,47,11/6/2019 16:52,R_2e9h2yDUhHWatJZ,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN
4,11/6/2019 18:38,11/6/2019 19:09,1862,11/6/2019 19:09,R_OrhB1M1BP1Aodr3,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,NaN,I don't have any suggestions. I believe my SI ...,No suggestions.,NaN,No suggestions.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,4/16/2025 15:40,4/16/2025 15:43,151,4/16/2025 15:43,R_5F2Rv9pOBWB3d3d,CHEM_36053_Results_Spring_2025,Strongly agree,Strongly agree,Strongly agree,NaN,...,NaN,Strongly agree,Strongly agree,Strongly agree,NaN,We would work together to complete the problem...,My SI leader was very competent and helpful. N...,NaN,NaN,SI has helped me in every semester of Chemistr...
34511,4/16/2025 15:54,4/16/2025 15:56,159,4/16/2025 15:56,R_5pmtyh6PJtkfD02,CHEM_36053_Results_Spring_2025,Strongly agree,Strongly agree,Strongly agree,NaN,...,NaN,Strongly agree,Strongly agree,Strongly agree,NaN,Working with other people has helped me by all...,I think my SI leader was great! Very knowledge...,NaN,NaN,No quizzes. I think attendance is what matters...
34512,4/16/2025 8:51,4/16/2025 16:02,25890,4/16/2025 16:02,R_7QNzZxoQkHhezbr,CHEM_36053_Results_Spring_2025,Moderately agree,Strongly agree,Strongly agree,NaN,...,NaN,Strongly agree,Strongly agree,Strongly agree,NaN,very nice!,NaN,NaN,NaN,NaN
34513,4/16/2025 16:11,4/16/2025 16:13,137,4/16/2025 16:13,R_3XpqiXNOLANRWly,CHEM_36053_Results_Spring_2025,Moderately agree,Strongly agree,Strongly agree,NaN,...,NaN,Strongly agree,Strongly agree,Strongly agree,NaN,We didnât collaborate a whole lot tbh but it...,none she did amazing \n,NaN,NaN,none its pretty well done


In [14]:
# source_file can be broken to extract important metadata about the courses. 
def parse_source_file(source_file):
    parts = source_file.split('_')
    
    # Discipline is always first.
    discipline = parts[0]
    
    # Course code is always second.
    course_code = parts[1]
    
    # Check if 'Online' is in the filename.
    is_online = 'Online' in source_file
    
    # Find semester and year.
    # Look for Fall/Spring in parts.
    semester = None
    year = None
    
    for i, part in enumerate(parts):
        if part in ['Fall', 'Spring']:
            semester = part
            # Year should be next part, consistently.
            if i + 1 < len(parts):
                year = parts[i + 1]
            break
    
    return {
        'discipline': discipline,
        'course_code': course_code,
        'is_online': is_online,
        'semester': semester,
        'year': int(year) if year else None
    }

# Apply parsing to create new columns.
parsed_data = consolidated_df['source_file'].apply(parse_source_file).apply(pd.Series)

# Add new columns to consolidated_df.
consolidated_df['Discipline'] = parsed_data['discipline']
consolidated_df['Course_Code'] = parsed_data['course_code']
consolidated_df['Is_Online'] = parsed_data['is_online']
consolidated_df['Semester'] = parsed_data['semester']
consolidated_df['Year'] = parsed_data['year']

# Verify the parsing worked.
print("New columns created:")
print(f"Disciplines: {consolidated_df['Discipline'].value_counts().to_dict()}")
print(f"Online courses: {consolidated_df['Is_Online'].value_counts().to_dict()}")
print(f"Semesters: {consolidated_df['Semester'].value_counts().to_dict()}")
print(f"Years: {sorted(consolidated_df['Year'].unique())}")

New columns created:
Disciplines: {'CHEM': 19605, 'BIOL': 6267, 'ACCT': 4334, 'ECON': 3749, 'NURS': 158, 'WCOB': 135, 'EXSC': 108, 'CHEG': 86, 'MEEG': 30, 'UNIV': 21, 'MATH': 19, 'MEGG': 3}
Online courses: {False: 33853, True: 662}
Semesters: {'Fall': 18293, 'Spring': 16222}
Years: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


In [15]:
# Create encoding mappings
agreement_scale = {
    'Strongly agree': 5,
    'Moderately agree': 4,
    'Neither disagree nor agree': 3,
    'Moderately disagree': 2,
    'Strongly disagree': 1,
    'Unable to judge': None
}

helpfulness_scale = {
    'Extremely helpful': 5,
    'Very helpful': 4,
    'Moderately helpful': 3,
    'Slightly helpful': 2,
    'Not at all helpful': 1,
    'Unable to judge': None
}

# Combine both mappings
rating_encoding = {**agreement_scale, **helpfulness_scale}

# Apply encoding to all R_ columns
rating_cols_consolidated = [c for c in consolidated_df.columns if c.startswith('R_')]

for col in rating_cols_consolidated:
    # Create new encoded column
    consolidated_df[col + '_encoded'] = consolidated_df[col].map(rating_encoding)

In [16]:
consolidated_df

,Start Date,End Date,Duration (in seconds),Recorded Date,Response ID,source_file,R_Blackboard_Resources,R_Collaborative_Activities,R_Concept_Discussion_Understanding,R_Dropin_Tutoring,...,R_Leader_Helps_Understanding_encoded,R_Leader_Knows_Class_Content_encoded,R_Leader_Subject_Competence_encoded,R_Leader_Willing_To_Help_encoded,R_Online_Active_Participation_encoded,R_Online_Engaging_Content_encoded,R_Opportunity_Ask_Questions_encoded,R_Overall_Session_Helpfulness_encoded,R_Practice_Problems_encoded,R_Weekly_Emails_encoded
0,11/6/2019 9:03,11/6/2019 9:03,29,11/6/2019 9:03,R_2QlC06WEVwEXtjQ,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,NaN
1,11/6/2019 10:06,11/6/2019 10:08,88,11/6/2019 10:08,R_VPgtpoWlVBH0E81,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,5.0,5.0,4.0,3.0,5.0,5.0,1.0,4.0,4.0,NaN
2,11/6/2019 12:18,11/6/2019 12:18,35,11/6/2019 12:18,R_sHdb8bWznRjZ6yB,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Moderately agree,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,NaN,4.0,5.0,NaN
3,11/6/2019 16:51,11/6/2019 16:52,47,11/6/2019 16:52,R_2e9h2yDUhHWatJZ,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN
4,11/6/2019 18:38,11/6/2019 19:09,1862,11/6/2019 19:09,R_OrhB1M1BP1Aodr3,ACCT_2013_Online_Results_Fall_2019,NaN,NaN,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,4/16/2025 15:40,4/16/2025 15:43,151,4/16/2025 15:43,R_5F2Rv9pOBWB3d3d,CHEM_36053_Results_Spring_2025,Strongly agree,Strongly agree,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,NaN,NaN,5.0,5.0,5.0,NaN
34511,4/16/2025 15:54,4/16/2025 15:56,159,4/16/2025 15:56,R_5pmtyh6PJtkfD02,CHEM_36053_Results_Spring_2025,Strongly agree,Strongly agree,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,NaN,NaN,5.0,5.0,5.0,NaN
34512,4/16/2025 8:51,4/16/2025 16:02,25890,4/16/2025 16:02,R_7QNzZxoQkHhezbr,CHEM_36053_Results_Spring_2025,Moderately agree,Strongly agree,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,NaN,NaN,5.0,5.0,5.0,NaN
34513,4/16/2025 16:11,4/16/2025 16:13,137,4/16/2025 16:13,R_3XpqiXNOLANRWly,CHEM_36053_Results_Spring_2025,Moderately agree,Strongly agree,Strongly agree,NaN,...,5.0,5.0,5.0,5.0,NaN,NaN,5.0,5.0,5.0,NaN
